# smog usage example

This notebook demonstrates the same workflow as the integration test using the sectional mass input data.

In [ ]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists():
            return candidate
    raise RuntimeError("Could not locate repository root (pyproject.toml not found).")

repo_root = find_repo_root(Path.cwd().resolve())
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from smog import (
    point_mass_cloud_from_mass_props,
    mass_cloud_to_dataframe,
    write_ply,
    write_rbd_pointMasses,
)

csv_path = repo_root / "tests" / "resources" / "sectional_masses.csv"
masses = pd.read_csv(csv_path)
masses.head()

In [ ]:
def breadth_at_x(breadth: float, x: float) -> float:
    if x < 10.0 or x > 130.0:
        return 0.75 * breadth
    return breadth

breadth = 11.75

frames = []
all_pos = []
all_m = []

for section in masses.to_dict("records"):
    pos, m = point_mass_cloud_from_mass_props(
        total_mass=float(row["m"]),
        cog_xyz=(float(row["cog_x"]), float(row["cog_y"]), float(row["cog_z"])),
        ixx=float(row["Ixx"]),
        iyy=float(row["Iyy"]),
        izz=float(row["Izz"]),
        n_points=20 * 6 * 6,
        span_xyz=(
            float(row["xend"] - row["xstart"]),
            breadth_at_x(breadth, float(row["cog_x"])),
            2.0 * float(row["cog_z"]),
        ),
    )

    frames.append(mass_cloud_to_dataframe(pos, m))
    all_pos.append(pos)
    all_m.append(m)

mass_cloud_df = pd.concat(frames, ignore_index=True)
pos_cloud = np.vstack(all_pos)
m_cloud = np.concatenate(all_m)

mass_cloud_df.head()

In [ ]:
out_dir = repo_root / "tmp" / "notebook_outputs"
out_dir.mkdir(parents=True, exist_ok=True)

rbd_path = out_dir / "point_masses.txt"
ply_path = out_dir / "point_masses.ply"

write_rbd_pointMasses(mass_cloud_df, rbd_path)
write_ply(ply_path, pos_cloud, m_cloud)

print(f"Wrote: {rbd_path}")
print(f"Wrote: {ply_path}")

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection="3d")
ax.scatter(pos_cloud[:, 0], pos_cloud[:, 1], pos_cloud[:, 2], s=1)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("SMOG point-mass cloud")
plt.tight_layout()
plt.show()